In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib widget
from scipy import ndimage
from scipy.stats import poisson
import os
import imageio
import damask
from PIL import Image

In [2]:
def plot(plane, nu_k, J_pos, J_neg, l, count):
    fig = plt.figure(figsize=(4,4))
    ax = fig.add_subplot(1, 1, 1)
    slipped_area = ax.imshow(plane,cmap='Grays')
    plt.xlim([-0.5,l-0.5])
    plt.ylim([-0.5,l-0.5])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['top']   .set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'] .set_visible(False)
    ax.spines['left']  .set_visible(False)
    plt.savefig(f'GIF/{nu_k}_{J_pos}_{J_neg}_{l}/{count}.png')
    plt.close()

In [3]:
def plot_pixel(plane, nu_k, J_pos, J_neg, l, count):
    filename = f'GIF/{nu_k}_{J_pos}_{J_neg}_{l}/{count}.png'
    MAX = plane.max()
    MIN = plane[plane>0].min()
    grayscale = np.linspace(0,0.8,num=MAX-MIN+1,endpoint=False)
    cmap = damask.Colormap(np.tile(grayscale, (3,1)).T,'name')
    cmap.shade(np.flipud(plane),bounds=[MIN,MAX],gap=0).save(filename)
    img = Image.open(filename)
    img.resize((500,500), Image.NEAREST).save(filename)

In [ ]:
def glide_plot(nu_k  =    1 ,   #lateral expansion speed normalized by kink-pair width, per second
               J_pos =    1.,   #nucleate rate of positive kink pairs on the whole dislocation line, per second
               J_neg =    1.,   #nucleate rate of negative kink pairs on the whole dislocation line, per second, need to be smaller than J_pos
               l     =   10 ,   #dislocation segment length normalized by kink-pair width, 1
               MAX   =  100 ,   #max number of positive nucleation events before cut off, 1
               tool  =  plot,   #plot tool
              ):
    #simulation time interval set as the time required for kink to move 1 unit when v_k is not zero, otherwise set as nucleation time
    delta_t = 1./nu_k if nu_k != 0 else 1./J_pos

    #initialization
    plane     = np.zeros((l,l), dtype=int) #simulation window
    plane_GIF = np.zeros((l,l), dtype=int)
    plane[0:3]     = 1  #0, unslipped; >=1, slipped
    plane_GIF[0:3] = 1
    count=0
    timestep = 0
    Nkp_pos = 0 #number of positive kink pair nucleation events
    os.system(f'mkdir GIF/{nu_k}_{J_pos}_{J_neg}_{l}')
    os.system(f'rm GIF/{nu_k}_{J_pos}_{J_neg}_{l}/*png')
    tool(plane_GIF, nu_k, J_pos, J_neg, l, count)
    while Nkp_pos < MAX:
        #time increment
        timestep+=1

        #lateral expansion
        if np.any(plane != plane[:,[0]]): #otherwise, the dislocation is a straight line
            plane = ndimage.binary_dilation(plane, structure=np.array([[0,0,0],[1,1,1],[0,0,0]],dtype=int)).astype(plane.dtype)
            plane_GIF = plane_GIF + plane
            count+=1
            tool(plane_GIF, nu_k, J_pos, J_neg, l, count)

        #kp nucleation
        New_kp_pos = int(poisson.ppf(np.random.uniform(), J_pos*delta_t))
        New_kp_neg = int(poisson.ppf(np.random.uniform(), J_neg*delta_t))
        Nkp_pos += New_kp_pos
        nuc_list = np.concatenate([np.ones(New_kp_pos,dtype=int), np.zeros(New_kp_neg,dtype=int)]) #all the nucleation events to happen in this time step
        np.random.shuffle(nuc_list) #randomize the event sequence
        for kp in nuc_list:
            pos_L = np.random.randint(0,l) #generate a random position along the segment
            pos_h = np.where(plane[:,pos_L]==(1-kp))[0][kp-1] #the opposite unit along that vertical line that's closest to the dislocation line
            plane[pos_h,pos_L] = kp #new kpz
            plane_GIF = plane_GIF + plane if kp==1 else plane_GIF * plane
            count+=1
            tool(plane_GIF, nu_k, J_pos, J_neg, l, count)

    #GIF
    film = []
    for n in range(count):
        film.append(imageio.v2.imread(f'GIF/{nu_k}_{J_pos}_{J_neg}_{l}/{n}.png'))
    imageio.mimsave(f'GIF/{nu_k}_{J_pos}_{J_neg}_{l}/{nu_k}_{J_pos}_{J_neg}_{l}.gif', # output gif
                    film,          # array of input frames
                    #fps = 5,      # optional: frames per second
                   )

In [165]:
glide_plot(nu_k  =  20,
           J_pos =  1,
           J_neg =  0.6,
           l     =  50,
           MAX   =  10,
           tool  = plot_pixel)

mkdir: GIF/20_1_0.6_50: File exists
